# Can skip, files saved

In [1]:
# Github for getting data
#https://github.com/Feng-CityUHK/EquityCharacteristics/blob/master/README.md
import pandas as pd
import numpy as np
import wrds
from pandas.tseries.offsets import *
import pickle as pkl

In [3]:
conn = wrds.Connection()

Enter your WRDS username [rguo]:tonyguo
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [7]:
conn.list_tables(library='comp')

['aco_amda',
 'aco_imda',
 'aco_indfnta',
 'aco_indfntq',
 'aco_indfntytd',
 'aco_indsta',
 'aco_indstq',
 'aco_indstytd',
 'aco_notesa',
 'aco_notesq',
 'aco_notessa',
 'aco_notesytd',
 'aco_pnfnda',
 'aco_pnfndq',
 'aco_pnfndytd',
 'aco_pnfnta',
 'aco_pnfntq',
 'aco_pnfntytd',
 'aco_transa',
 'aco_transq',
 'aco_transsa',
 'aco_transytd',
 'adsprate',
 'asec_amda',
 'asec_imda',
 'asec_notesa',
 'asec_notesq',
 'asec_transa',
 'asec_transq',
 'bank_aacctchg',
 'bank_adesind',
 'bank_afnd1',
 'bank_afnd2',
 'bank_afnddc1',
 'bank_afnddc2',
 'bank_afntind',
 'bank_funda',
 'bank_funda_fncd',
 'bank_fundq',
 'bank_fundq_fncd',
 'bank_iacctchg',
 'bank_idesind',
 'bank_ifndq',
 'bank_ifndytd',
 'bank_ifntq',
 'bank_ifntytd',
 'bank_names',
 'bank_namesq',
 'chars',
 'co_aacctchg',
 'co_aaudit',
 'co_adesind',
 'co_adjfact',
 'co_afnd1',
 'co_afnd2',
 'co_afnddc1',
 'co_afnddc2',
 'co_afntind1',
 'co_afntind2',
 'co_ainvval',
 'co_amkt',
 'co_busdescl',
 'co_cotype',
 'co_filedate',
 'co_

In [ ]:
library_name = "comp"  # Replace with the name of the library you're interested in
query = f"""
    SELECT table_name, table_description
    FROM information_schema.tables
    WHERE table_schema = '{library_name}'
"""
tables_info = conn.raw_sql(query)

In [ ]:
# Compustat block
comp = conn.raw_sql("""
                    /*header info*/
                    select c.gvkey, f.cusip, f.datadate, f.fyear, c.cik, substr(c.sic,1,2) as sic2, c.sic, c.naics,
                    
                    /*firm variables*/
                    /*income statement*/
                    f.sale, f.revt, f.cogs, f.xsga, f.dp, f.xrd, f.xad, f.ib, f.ebitda,
                    f.ebit, f.nopi, f.spi, f.pi, f.txp, f.ni, f.txfed, f.txfo, f.txt, f.xint,
                    
                    /*CF statement and others*/
                    f.capx, f.oancf, f.dvt, f.ob, f.gdwlia, f.gdwlip, f.gwo, f.mib, f.oiadp, f.ivao,
                    
                    /*assets*/
                    f.rect, f.act, f.che, f.ppegt, f.invt, f.at, f.aco, f.intan, f.ao, f.ppent, f.gdwl, f.fatb, f.fatl,
                    
                    /*liabilities*/
                    f.lct, f.dlc, f.dltt, f.lt, f.dm, f.dcvt, f.cshrc, 
                    f.dcpstk, f.pstk, f.ap, f.lco, f.lo, f.drc, f.drlt, f.txdi,
                    
                    /*equity and other*/
                    f.ceq, f.scstkc, f.emp, f.csho, f.seq, f.txditc, f.pstkrv, f.pstkl, f.np, f.txdc, f.dpc, f.ajex,
                    
                    /*market*/
                    abs(f.prcc_f) as prcc_f
                    
                    from comp.funda as f
                    left join comp.company as c
                    on f.gvkey = c.gvkey
                    
                    /*get consolidated, standardized, industrial format statements*/
                    where f.indfmt = 'INDL' 
                    and f.datafmt = 'STD'
                    and f.popsrc = 'D'
                    and f.consol = 'C'
                    and f.datadate >= '01/01/2024'
                    """)

In [ ]:
comp.to_csv('compustat.csv')

In [ ]:
# CRSP block
crsp = conn.raw_sql("""
                      select a.prc, a.ret, a.retx, a.shrout, a.vol, a.cfacpr, a.cfacshr, a.date, a.permno, a.permco,
                      b.ticker, b.ncusip, b.shrcd, b.exchcd
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date >= '01/01/1959'
                      and b.exchcd between 1 and 3
                      """)

In [ ]:
crsp.to_csv('crsp_data.csv')

In [ ]:
# CCM block
ccm = conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """)

In [ ]:
ccm.to_csv('ccm_data.csv')

In [ ]:
# Compustat quarterly
comp_q = conn.raw_sql("""
                    /*header info*/
                    select c.gvkey, f.cusip, f.datadate, f.fyearq,  substr(c.sic,1,2) as sic2, c.sic, f.fqtr, f.rdq,

                    /*income statement*/
                    f.ibq, f.saleq, f.txtq, f.revtq, f.cogsq, f.xsgaq, f.revty, f.cogsy, f.saley,

                    /*balance sheet items*/
                    f.atq, f.actq, f.cheq, f.lctq, f.dlcq, f.ppentq, f.ppegtq,

                    /*others*/
                    abs(f.prccq) as prccq, abs(f.prccq)*f.cshoq as mveq_f, f.ceqq, f.seqq, f.pstkq, f.ltq,
                    f.pstkrq, f.gdwlq, f.intanq, f.mibq, f.oiadpq, f.ivaoq,
                    
                    /* v3 my formula add*/
                    f.ajexq, f.cshoq, f.txditcq, f.npq, f.xrdy, f.xrdq, f.dpq, f.xintq, f.invtq, f.scstkcy, f.niq,
                    f.oancfy, f.dlttq, f.rectq, f.acoq, f.apq, f.lcoq, f.loq, f.aoq

                    from comp.fundq as f
                    left join comp.company as c
                    on f.gvkey = c.gvkey

                    /*get consolidated, standardized, industrial format statements*/
                    where f.indfmt = 'INDL' 
                    and f.datafmt = 'STD'
                    and f.popsrc = 'D'
                    and f.consol = 'C'
                    and f.datadate >= '01/01/1959'
                    """)

In [ ]:
comp_q.to_csv('comp_quarterly.csv')

In [ ]:
# Momentum
crsp_mom = conn.raw_sql("""
                        select permno, date, ret, retx, prc, shrout, vol
                        from crsp.msf
                        where date >= '01/01/1959'
                        """)

In [ ]:
crsp_mom.to_csv('momentum.csv')

# Construct features

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
import pickle as pkl
import pyarrow.feather as feather
# from functions import *

functions.py contains ffi149, ffi30, ffi12, fillna_atq, fillna_ind, fillna_all, standardize

In [2]:
comp = pd.read_csv('compustat.csv')
comp_q = pd.read_csv('comp_quarterly.csv')
crsp = pd.read_csv('crsp_data.csv')
ccm = pd.read_csv('ccm_data.csv')
crsp_mom = pd.read_csv('momentum.csv')

C:\Users\rguo\AppData\Local\Temp\ipykernel_29080\3670173509.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  comp = pd.read_csv('compustat.csv')


In [3]:
comp.columns

Index(['Unnamed: 0', 'gvkey', 'cusip', 'datadate', 'fyear', 'cik', 'sic2',
       'sic', 'naics', 'sale', 'revt', 'cogs', 'xsga', 'dp', 'xrd', 'xad',
       'ib', 'ebitda', 'ebit', 'nopi', 'spi', 'pi', 'txp', 'ni', 'txfed',
       'txfo', 'txt', 'xint', 'capx', 'oancf', 'dvt', 'ob', 'gdwlia', 'gdwlip',
       'gwo', 'mib', 'oiadp', 'ivao', 'rect', 'act', 'che', 'ppegt', 'invt',
       'at', 'aco', 'intan', 'ao', 'ppent', 'gdwl', 'fatb', 'fatl', 'lct',
       'dlc', 'dltt', 'lt', 'dm', 'dcvt', 'cshrc', 'dcpstk', 'pstk', 'ap',
       'lco', 'lo', 'drc', 'drlt', 'txdi', 'ceq', 'scstkc', 'emp', 'csho',
       'seq', 'txditc', 'pstkrv', 'pstkl', 'np', 'txdc', 'dpc', 'ajex',
       'prcc_f'],
      dtype='object')

In [7]:
comp['datadate']

0         12/31/1984
1         12/31/1985
2         12/31/1986
3         12/31/1987
4          6/30/1988
             ...    
558162    12/31/2022
558163    12/31/2022
558164    12/31/2022
558165    12/31/2022
558166    12/31/2023
Name: datadate, Length: 558167, dtype: object

In [ ]:
# 1 month momentum
crsp_mom['mom1m'] = crsp_mom['ret']

# Dollar trading volume
crsp_mom['vol_l2'] = crsp_mom.groupby(['permno'])['vol'].shift(2)
crsp_mom['prc_l2'] = crsp_mom.groupby(['permno'])['prc'].shift(2)
crsp_mom['dolvol'] = np.log(crsp_mom['vol_l2']*crsp_mom['prc_l2']).replace([np.inf, -np.inf], np.nan)

# share turnover
crsp_mom['vol_l1'] = crsp_mom.groupby(['permno'])['vol'].shift(1)
crsp_mom['vol_l3'] = crsp_mom.groupby(['permno'])['vol'].shift(3)
crsp_mom['turn'] = ((crsp_mom['vol_l1']+crsp_mom['vol_l2']+crsp_mom['vol_l3'])/3)/crsp_mom['shrout']

# raw annual data
data_rawa = pd.merge(crsp2, ccm2, how='inner', on=['permno', 'jdate'])

